In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [6]:
data=pd.read_csv("/Users/ishaankadle/Downloads/train.csv")
data_test=pd.read_csv("/Users/ishaankadle/Downloads/test.csv")

df=data.copy()
df_test=data_test.copy()

In [7]:
pd.get_dummies(df.Gender,drop_first=True).astype(int)
pd.get_dummies(df_test.Gender,drop_first=True).astype(int)
df["Gender_Encoded"]=pd.get_dummies(df.Gender,drop_first=True).astype(int)
df_test["Gender_Encoded"]=pd.get_dummies(df_test.Gender,drop_first=True).astype(int)

In [8]:
city_encoded=pd.get_dummies(df['City_Category'], drop_first=True).astype(int)
city_encoded_test=pd.get_dummies(df_test['City_Category'], drop_first=True).astype(int)

df = pd.concat([df, city_encoded], axis=1)
df_test = pd.concat([df_test, city_encoded_test], axis=1)

In [9]:
df=df.drop(columns=['Gender','City_Category'])
df_test=df_test.drop(columns=['Gender','City_Category'])

In [10]:
age_encoder_1={
    '0-17': 8.5,
    '18-25': 21.5,
    '26-35': 30.5,
    '36-45': 40.5,
    '46-50': 48,
    '51-55': 53,
    '55+': 55
}

df['Age'] = df['Age'].map(age_encoder_1)
df_test['Age'] = df_test['Age'].map(age_encoder_1)

In [12]:
stay_encoder={
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4+': 4
}
df['Stay'] = df['Stay_In_Current_City_Years'].map(stay_encoder)
df_test['Stay'] = df_test['Stay_In_Current_City_Years'].map(stay_encoder)

In [13]:
df=df.drop(columns=['Stay_In_Current_City_Years'])
df_test=df_test.drop(columns=['Stay_In_Current_City_Years'])

In [16]:
df.columns

Index(['User_ID', 'Product_ID', 'Age', 'Occupation', 'Marital_Status',
       'Product_Category_1', 'Product_Category_2', 'Product_Category_3',
       'Purchase', 'Gender_Encoded', 'B', 'C', 'Stay'],
      dtype='object')

In [23]:
df.dtypes

User_ID                 int64
Product_ID             object
Age                   float64
Occupation              int64
Marital_Status          int64
Product_Category_1      int64
Product_Category_2    float64
Product_Category_3    float64
Purchase                int64
Gender_Encoded          int64
B                       int64
C                       int64
Stay                    int64
dtype: object

In [25]:
df['Product_Category_2'].fillna(df.Product_Category_2.mean(),inplace=True)
df_test['Product_Category_2'].fillna(df_test.Product_Category_2.mean(),inplace=True)

df['Product_Category_3'].fillna(0,inplace=True)
df_test['Product_Category_3'].fillna(0,inplace=True)


/var/folders/mm/0szfp9_x6bb9bgc1tpkndv340000gn/T/ipykernel_54495/3893328079.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Product_Category_2'].fillna(df.Product_Category_2.mean(),inplace=True)
/var/folders/mm/0szfp9_x6bb9bgc1tpkndv340000gn/T/ipykernel_54495/3893328079.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which 

In [27]:
df.columns

Index(['User_ID', 'Product_ID', 'Age', 'Occupation', 'Marital_Status',
       'Product_Category_1', 'Product_Category_2', 'Product_Category_3',
       'Purchase', 'Gender_Encoded', 'B', 'C', 'Stay'],
      dtype='object')

In [29]:
X_train = df[['Age', 'Occupation', 'Marital_Status',
       'Gender_Encoded', 'B', 'C', 'Stay', 'Product_Category_1',
             'Product_Category_2','Product_Category_3']]

In [31]:
Y_train = df['Purchase'] 

In [33]:
X_test = df_test[['Age', 'Occupation', 'Marital_Status',
       'Gender_Encoded', 'B', 'C', 'Stay', 'Product_Category_1',
             'Product_Category_2','Product_Category_3',]]

In [35]:
import numpy as np
from sklearn.ensemble import (
    HistGradientBoostingRegressor,
    StackingRegressor,
    RandomForestRegressor
)
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV


In [37]:
rf_model = RandomForestRegressor() 
hgb_model = HistGradientBoostingRegressor(learning_rate= 0.1,max_depth=10, max_iter= 1000, min_samples_leaf= 5) 
lgbm_model = LGBMRegressor(learning_rate= 0.1, max_depth= 20, n_estimators= 500, num_leaves= 100)
xgb_model = XGBRegressor(colsample_bytree=0.7,learning_rate=0.05,max_depth=10, n_estimators= 500) 
catboost_model = CatBoostRegressor(depth= 10, iterations= 1000, l2_leaf_reg= 5, learning_rate= 0.1, silent=True)  

In [ ]:
#Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 500],            
    'max_depth': [10, 20, 30, None],            
    'min_samples_split': [2, 5, 10],            
    'min_samples_leaf': [1, 2, 4],              
    'max_features': ['auto', 'sqrt', 'log2'],  
    'bootstrap': [True, False]                 
}

grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, 
                              cv=3, scoring='neg_mean_squared_error', 
                              verbose=2, n_jobs=-1)

grid_search_rf.fit(X_train, Y_train)

print("Best parameters for RandomForestRegressor: ", grid_search_rf.best_params_)

Fitting 3 folds for each of 648 candidates, totalling 1944 fits
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=500; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=200; t

/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=500; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=500; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_fe

In [266]:
#High Gradient Boost
param_grid_hgb = {
    'learning_rate': [0.1, 0.125, 0.15],  
    'max_iter': [500, 700, 1000],  
    'max_depth': [10,12,15],  
    'min_samples_leaf': [5, 7, 10] 
}

grid_search_hgb = GridSearchCV(estimator=hgb_model, param_grid=param_grid_hgb, 
                               cv=3, scoring='neg_mean_squared_error', 
                               verbose=2, n_jobs=-1)

grid_search_hgb.fit(X_train, Y_train)

print("Best parameters for HistGradientBoostingRegressor: ", grid_search_hgb.best_params_)


Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best parameters for HistGradientBoostingRegressor:  {'learning_rate': 0.15, 'max_depth': 12, 'max_iter': 1000, 'min_samples_leaf': 10}


In [271]:
#Light BGM (Run 1 time)
param_grid_lgbm = {
    'n_estimators': [500, 700, 1000],  
    'learning_rate': [0.1, 0.125,0.15],  
    'num_leaves': [100, 125, 150],  
    'max_depth': [15, 20, 25]  
}


grid_search_lgbm = GridSearchCV(estimator=lgbm_model, param_grid=param_grid_lgbm, 
                                cv=3, scoring='neg_mean_squared_error', 
                                verbose=2, n_jobs=-1)

grid_search_lgbm.fit(X_train, Y_train)

print("Best parameters for LGBMRegressor: ", grid_search_lgbm.best_params_)


Fitting 3 folds for each of 81 candidates, totalling 243 fits


KeyboardInterrupt: 

In [ ]:
#XGB Regressor 

param_grid_xgb = {
    'n_estimators': [500,700,1000],  
    'learning_rate': [0.025, 0.05, 0.075],  
    'max_depth': [10, 12, 15 ], 
    'colsample_bytree': [0.5, 0.7, 0.9] 
}

grid_search_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, 
                               cv=3, scoring='neg_mean_squared_error', 
                               verbose=2, n_jobs=-1)

grid_search_xgb.fit(X_train, Y_train)

print("Best parameters for XGBRegressor: ", grid_search_xgb.best_params_)


In [ ]:
#Cat Boost Regressor 

param_grid_catboost = {
    'iterations': [1000, 1250, 1500],  
    'depth': [10, 12, 15],  
    'learning_rate': [0.1, 0.125, 0.15], 
    'l2_leaf_reg': [5, 7, 10]  
}


grid_search_catboost = GridSearchCV(estimator=catboost_model, param_grid=param_grid_catboost, 
                                    cv=3, scoring='neg_mean_squared_error', 
                                    verbose=2, n_jobs=-1)

grid_search_catboost.fit(X_train, Y_train)

print("Best parameters for CatBoostRegressor: ", grid_search_catboost.best_params_)

In [39]:
stacking_model = StackingRegressor(
    estimators=[
        ('rf', rf_model),  
        ('hgb', hgb_model),
        ('lgbm', lgbm_model),
        ('xgb', xgb_model),
        ('catboost', catboost_model)
    ],
    final_estimator=LinearRegression()
)


stacking_model.fit(X_train, Y_train)
df_test['Purchase'] = stacking_model.predict(X_test)  

df_test['Purchase'] = np.round(df_test['Purchase'])

In [40]:
df_test[['User_ID', 'Product_ID', 'Purchase']].to_csv('/Users/ishaankadle/Downloads/solutionaai.csv', index=False)